In [ ]:
import numpy as np
import plotly.express as px
import os
import nibabel as nib
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
PI_UNICODE = "\u03C0"

### Figure

In [ ]:
fname_mag_e1 = os.path.join("fmap", "BR_e1_M.nii")
fname_phase_e1 = os.path.join("fmap", "BR_e1_ph.nii")
fname_phase_e2 = os.path.join("fmap", "BR_e2_ph.nii")
fname_phase_e3 = os.path.join("fmap", "BR_e3_ph.nii")
fname_phase_e4 = os.path.join("fmap", "BR_e4_ph.nii")
fname_mask = os.path.join("fmap", "mask.nii.gz")
fname_fmap = os.path.join("fmap", "fmap.nii")

nii_mag_e1 = nib.load(fname_mag_e1)
nii_phase_e1 = nib.load(fname_phase_e1)
nii_phase_e2 = nib.load(fname_phase_e2)
nii_phase_e3 = nib.load(fname_phase_e3)
nii_phase_e4 = nib.load(fname_phase_e4)
nii_mask = nib.load(fname_mask)
nii_fmap = nib.load(fname_fmap)

In [ ]:
def complex_difference(phase1, phase2):
    """ Calculates the complex difference between 2 phase arrays (phase2 - phase1)

    Args:
        phase1 (numpy.ndarray): Array containing phase data in radians
        phase2 (numpy.ndarray): Array containing phase data in radians. Must be the same shape as phase1.

    Returns:
        numpy.ndarray: The difference in phase between each voxels of phase2 and phase1 (phase2 - phase1)
    """

    # Calculate phasediff using complex difference
    comp_0 = np.ones_like(phase1) * np.exp(-1j * phase1)
    comp_1 = np.ones_like(phase2) * np.exp(1j * phase2)
    return np.angle(comp_0 * comp_1)

In [ ]:
def plot_2_echo_fmap(phase1, phase2, echotime1, echotime2):
    phase_diff = complex_difference(phase1, phase2)
    fmap = phase_diff / (echotime2 - echotime1) / 2 / math.pi
    n=4
    # Attempt at subplots
    fig = make_subplots(rows=1, cols=n, shared_xaxes=False, horizontal_spacing=0.1, subplot_titles=("Phase 1", "Phase 2", "Phase difference", "B0 field map"), specs=[[{"type": "Heatmap"}, {"type": "Heatmap"}, {"type": "Heatmap"}, {"type": "Heatmap"}]], )
    
    fig.add_trace(go.Heatmap(z=np.rot90(phase1, k=-1), colorscale='gray', colorbar_x=1/n - 0.05, zmin=-math.pi, zmax=math.pi,
                             colorbar=dict(title="Rad",
                                           titleside="top",
                                           tickmode="array",
                                           tickvals=[-math.pi, 0, math.pi-0.01],
                                           ticktext = [f"-{PI_UNICODE}", 0, f'{PI_UNICODE}'])), 1, 1)
    fig.add_trace(go.Heatmap(z=np.rot90(phase2, k=-1), colorscale='gray', colorbar_x=2/n - 0.02, zmin=-math.pi, zmax=math.pi,
                             colorbar=dict(title="Rad",
                                           titleside="top",
                                           tickmode="array",
                                           tickvals=[-math.pi, 0, math.pi-0.01],
                                           ticktext = [f"-{PI_UNICODE}", 0, f'{PI_UNICODE}'])), 1, 2)
    fig.add_trace(go.Heatmap(z=np.rot90(phase_diff, k=-1), colorscale='gray', colorbar_x=3/n - 0.02, zmin=-math.pi, zmax=math.pi,
                             colorbar=dict(title="Rad",
                                           titleside="top",
                                           tickmode="array",
                                           tickvals=[-math.pi, 0, math.pi-0.01],
                                           ticktext = [f"-{PI_UNICODE}", 0, f'{PI_UNICODE}'])), 1, 3)
    fig.add_trace(go.Heatmap(z=np.rot90(fmap, k=-1), colorscale='gray',
                             colorbar=dict(title="Hz",
                                           titleside="top")), 1, 4)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    
    fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
mask = nii_mask.get_fdata()[:-10,:,50]
phase3 = (nii_phase_e3.get_fdata()[:-10,:,50] / 4095 * 2 * math.pi - math.pi) * mask
phase4 = (nii_phase_e4.get_fdata()[:-10,:,50] / 4095 * 2 * math.pi - math.pi) * mask
echo_time3 = 0.009
echo_time4 = 0.013

plot_2_echo_fmap(phase3, phase4, echo_time3, echo_time4)

### Figure

In [ ]:
# Phase evolution though different echo times
mask = nii_mask.get_fdata()[:-10,:,50]
fmap = nii_fmap.get_fdata()[:-10,:,50] * mask  # [Hz]
phase1 = (nii_phase_e1.get_fdata()[:-10,:,50] / 4095 * 2 * math.pi - math.pi) * mask

echo_time1 = 0.00263
phase0 = phase1 - (0.00263 * (fmap * 2 * math.pi))

echo_times = np.linspace(0.001, 0.02, 20)
fig = go.Figure()
for echo_time in echo_times:
    phase = phase0 + (fmap * echo_time * 2 * math.pi)
    phase = np.angle(np.exp(1j*phase))
    fig.add_trace(go.Heatmap(z=np.rot90(phase, k=-1), colorscale='gray'))

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        label=echo_times[i],
        args=[{"visible": [False] * len(fig.data)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

fig.data[0].visible = True

sliders = [dict(
    active=19,
    currentvalue={"prefix": "Echo Time: "},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.update_layout(
    title=dict(text="Phase at different echo times", x=0.5)
)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.update_layout({"height": 500, "width": 500})
fig.show()

### Figure

In [ ]:

def get_circle(x, y, r):
    if x < 1 or y < 1 or r < 1:
        raise ValueError("Input parameters are too small")
        
    my_array = np.zeros([x,y])
    for i in range(x):
        for j in range(y):
            squared = (i-(x/2))**2 + (j-(y/2))**2
            h = np.sqrt(squared)
            if h < r:
                my_array[i,j] = 1
    return my_array

echo1 = get_circle(100, 100, 30) * -1
echo2 = get_circle(100, 100, 30) * 2
echo_time1 = 0.005
echo_time2 = 0.01

plot_2_echo_fmap(echo1, echo2, echo_time1, echo_time2)
